In [1]:
import builtins
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml import ExecutionConfiguration, MLVocab, DerivaSystemColumns, DatasetSpec, DerivaML
from deriva_ml.demo_catalog import create_demo_catalog, DemoML
from IPython.display import display, Markdown, JSON
import itertools
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
foo = 1

In [3]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


You are already logged in.


In [4]:
test_catalog = create_demo_catalog(hostname, domain_schema, create_features=True, create_datasets=True)
ml_instance = DemoML(hostname, test_catalog.catalog_id)
print(f'Creating catalog at {ml_instance.catalog_id}')

ml_instance.add_term(MLVocab.workflow_type, "Manual Workflow", description="Initial setup of Model File")
ml_instance.add_term(MLVocab.asset_type, "API_Model", description="Model for our API workflow")

api_workflow = ml_instance.create_workflow(
    name="Manual Workflow",
    workflow_type="Manual Workflow",
    description="A manual operation"
)

manual_execution = ml_instance.create_execution(ExecutionConfiguration( description="Sample Execution", workflow=api_workflow))

# Now lets create model configuration for our program.
model_file = manual_execution.asset_file_path("Execution_Asset",'modelfile.txt', asset_types="API_Model")
with builtins.open(model_file, "w") as fp:
    fp.write(f"My model")

# Now upload the file and retrieve the RID of the new asset from the returned results.
uploaded_assets = manual_execution.upload_execution_outputs()
training_model_rid = [a.asset_rid  for a  in uploaded_assets['deriva-ml/Execution_Asset'] if 'API_Model' in a.asset_types][0]

display(
    Markdown(f'## Training Model: {training_model_rid}'),
    JSON(ml_instance.retrieve_rid(training_model_rid))
)

TypeError: Workflow._get_python_script() missing 1 required positional argument: 'self'